In [ ]:
pip install requests beautifulsoup4 selenium

Download Library to interact with Youtube API

In [1]:
pip install google-api-python-client pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: C:\Users\yesha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


upgrade youtube api installation

In [2]:
pip uninstall google-api-python-client

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install google-api-python-client

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: C:\Users\yesha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


YOUTUBE API (VIEW, LIKE AND COMMMENT COUNT) with error [for cleaned and final csv file]

In [ ]:
import socket
import os
import time
import csv
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# ================
# Force IPv4 Only
# ================
original_getaddrinfo = socket.getaddrinfo

def getaddrinfo_ipv4_only(host, port, family=0, type=0, proto=0, flags=0):
    return original_getaddrinfo(host, port, socket.AF_INET, type, proto, flags)

socket.getaddrinfo = getaddrinfo_ipv4_only
socket.setdefaulttimeout(10)  # Set global timeout

# ================
# Bypass Proxy
# ================
os.environ['NO_PROXY'] = 'youtube.googleapis.com,googleapis.com'

# ================
# YouTube API Setup
# ================
API_KEY = "PUT API KEY HERE"
youtube = build("youtube", "v3", developerKey=API_KEY)

# ================
# Get Video Details with Retry Logic
# ================
def get_video_details(video_id, max_retries=3, delay=5):
    for attempt in range(max_retries):
        try:
            response = youtube.videos().list(
                part="statistics",
                id=video_id
            ).execute()

            if not response.get("items"):
                print(f"No data found for video ID {video_id}")
                return None, None, None, "No data returned"

            stats = response["items"][0]["statistics"]
            view_count = stats.get("viewCount", 0)
            like_count = stats.get("likeCount", 0)
            comment_count = stats.get("commentCount", 0)

            return int(view_count), int(like_count), int(comment_count), None

        except HttpError as e:
            error_reason = e.error_details[0]['reason'] if e.error_details else str(e)
            print(f"HTTP Error {e.resp.status} for video ID {video_id}: {error_reason}")
            if e.resp.status in [429, 500, 503]:  # Rate limit or server error
                print(f"Retrying in {delay}s...")
                time.sleep(delay)
            else:
                return None, None, None, error_reason
        except Exception as e:
            print(f"Unexpected error for video ID {video_id}: {e}")
            if "10060" in str(e):  # Connection timeout
                print(f"Connection timed out. Retrying in {delay}s...")
                time.sleep(delay)
            else:
                return None, None, None, str(e)

    return None, None, None, "Max retries exceeded"

# ================
# Process CSV File
# ================
def process_videos(input_csv, output_csv):
    df = pd.read_csv(input_csv)

    if 'Video Id' in df.columns:
        df = df.rename(columns={'Video Id': 'Video ID'})
    elif 'Video ID' not in df.columns:
        raise ValueError("Input CSV must contain a column named 'Video ID' or 'Video Id'.")

    results = []
    error_video_ids = []

    for index, row in df.iterrows():
        video_id = row['Video ID']
        print(f"Processing video ID: {video_id} (Index: {index})")

        view_count, like_count, comment_count, error = get_video_details(video_id)

        if error:
            error_video_ids.append(f"{video_id} | Reason: {error}")
        else:
            results.append({
                "Video ID": video_id,
                "View Count": view_count,
                "Like Count": like_count,
                "Comment Count": comment_count
            })

        time.sleep(0.5)  # Delay to avoid rate limiting

    results_df = pd.DataFrame(results)
    results_df.to_csv(output_csv, index=False)
    print(f"Results written to {output_csv}")

    error_file_name = "error_video_ID_engagement.txt"
    with open(error_file_name, "w") as error_file:
        for video_id in error_video_ids:
            error_file.write(video_id + "\n")
    print(f"List of error video IDs written to {error_file_name}")

# ================
# Configuration
# ================
input_csv = "Final_clustering/cleaned.csv"
output_csv = "Final_clustering/Results_Count_Final.csv"

process_videos(input_csv, output_csv)

Processing video ID: --8n6A8Q6M0 (Index: 0)
Processing video ID: -0FfyNZdHk0 (Index: 1)
Processing video ID: -1B7cVoZr1c (Index: 2)
Processing video ID: -4yu_13EHFM (Index: 3)
Processing video ID: -6FvAU9H98w (Index: 4)
Processing video ID: -7vF5F-1btE (Index: 5)
Processing video ID: -9ACKA4sQ4U (Index: 6)
Processing video ID: -9bfDHHneyU (Index: 7)
Processing video ID: -9hDaG3nJwg (Index: 8)
Processing video ID: -C5iB25BRsA (Index: 9)
Processing video ID: -GVbt3qdq70 (Index: 10)
Processing video ID: -JTkbbE_KP4 (Index: 11)
Processing video ID: -JlBNKlG3Yw (Index: 12)
Processing video ID: -KHOXPq_M-Q (Index: 13)
Processing video ID: -NkPxyvAqzA (Index: 14)
Processing video ID: -O-ZnlGZq-4 (Index: 15)
Processing video ID: -P_7E7gq6vs (Index: 16)
Processing video ID: -Pe-c-CHkIo (Index: 17)
Processing video ID: -Ts46vj5sEc (Index: 18)
Processing video ID: -U5-r29G-t0 (Index: 19)
Processing video ID: -WaDvcu7mcw (Index: 20)
Processing video ID: -Wgdk-C7Rb8 (Index: 21)
Processing video ID:

Extract the Suscribers Count for the Video ID

In [ ]:
import pandas as pd
import time
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# ================
# YouTube API Setup
# ================
API_KEY = "PUT API KEY HERE"  # <-- Replace with your actual API key
youtube = build("youtube", "v3", developerKey=API_KEY)

# ================
# Get Channel Info (Name & Subscribers) with Retry Logic
# ================
def get_channel_info(channel_id, max_retries=3, delay=5):
    for attempt in range(max_retries):
        try:
            response = youtube.channels().list(
                part="snippet,statistics",
                id=channel_id
            ).execute()
            if not response.get("items"):
                print(f"No data found for channel ID {channel_id}")
                return None, None, "No data returned"
            item = response["items"][0]
            channel_name = item["snippet"]["title"]
            sub_count_str = item["statistics"].get("subscriberCount")
            hidden = item["statistics"].get("hiddenSubscriberCount", False)
            if hidden:
                print(f"Subscriber count hidden for channel {channel_id}")
                sub_count = None
            else:
                sub_count = int(sub_count_str) if sub_count_str else None
            return channel_name, sub_count, None
        except HttpError as e:
            status = e.resp.status
            print(f"HTTP Error {status} for channel ID {channel_id}: {e}")
            if status in [429, 500, 503]:
                print(f"Retrying in {delay}s... (Attempt {attempt + 1}/{max_retries})")
                time.sleep(delay * (2 ** attempt))  # Exponential backoff
            else:
                return None, None, str(e)
        except Exception as e:
            print(f"Unexpected error for channel ID {channel_id}: {e}")
            time.sleep(delay)
    return None, None, "Max retries exceeded"

# ================
# Main Processing
# ================
def process_unique_channels(input_csv, output_csv):
    # Read the CSV
    df = pd.read_csv(input_csv)
    if 'Channel Id' not in df.columns:
        raise ValueError(f"Input CSV must contain 'Channel Id'. Found columns: {list(df.columns)}")
    unique_channels = df['Channel Id'].dropna().unique()
    print(f"Found {len(unique_channels)} unique channel IDs.")

    results = []
    errors = []

    for idx, channel_id in enumerate(unique_channels):
        print(f"[{idx+1}/{len(unique_channels)}] Processing Channel ID: {channel_id}")
        channel_name, sub_count, error = get_channel_info(channel_id)
        if error:
            print(f"  ❌ Error: {error}")
            errors.append(f"{channel_id} | Reason: {error}")
        else:
            print(f"  ✅ Channel: '{channel_name}' | Subs: {sub_count}")
        results.append({
            "Channel Id": channel_id,
            "Channel Name": channel_name,
            "Subscriber Count": sub_count
        })
        time.sleep(0.6)  # Be gentle with API quota

    # Save results
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_csv, index=False)
    print(f"✅ Subscriber data saved to: {output_csv}")

    # Save errors
    if errors:
        with open("error_channel_ID_subscribers.txt", "w") as f:
            for line in errors:
                f.write(line + "\n")
        print("📝 Error log saved to: error_channel_ID_subscribers.txt")
    else:
        print("🎉 No errors encountered!")

# ================
# Configuration
# ================
input_csv = "Final_clustering/cleaned-with-channels.csv"
output_csv = "Final_Clustering/Final_Suscribers_Count.csv"

process_unique_channels(input_csv, output_csv)

Found 929 unique channel IDs.
[1/929] Processing Channel ID: UCWm__g4cPmX-umorqM8VfWw
  ✅ Channel: 'Travel with Luke Damant' | Subs: 1620000
[2/929] Processing Channel ID: UCR-yUQRWEciSqQowtTDHskA
  ✅ Channel: 'Learn with Daryll' | Subs: 5250
[3/929] Processing Channel ID: UCUB6A2YY9soreMGNwDLh-zw
  ✅ Channel: 'Combat Arms Channel' | Subs: 589000
[4/929] Processing Channel ID: UCFvQ_bDaalb-1ZyCb48LCpA
  ✅ Channel: 'Chasing Kimberly' | Subs: 16500
[5/929] Processing Channel ID: UCixD9UbKvDxzGNiPC_fgHyA
  ✅ Channel: 'Paolo fromTOKYO' | Subs: 3620000
[6/929] Processing Channel ID: UC_hN8xs99S6jnMdYCuJgoJA
  ✅ Channel: 'Pinoy Planet' | Subs: 97800
[7/929] Processing Channel ID: UCufN1_2lxrI_C5JgbfU89ew
  ✅ Channel: ' 🌻Mia's Kitchen🌻' | Subs: 3480
[8/929] Processing Channel ID: UC-wdvwNBuVobJRbISwjA5Hw
  ✅ Channel: 'Alex Oh Reacts' | Subs: 179000
[9/929] Processing Channel ID: UCi9mzS8H-ez_30w5SCL2dmw
  ✅ Channel: 'JREKML' | Subs: 1210000
[10/929] Processing Channel ID: UChJZEuE4O45M6CbcYOS